In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
seed = 42
np.random.seed(seed)

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor

import xgboost

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix


%matplotlib inline

# Intro

One of the problems with using single decision trees and random forests is that, once I make a split, I can't go back and consider how another feature varies across the whole dataset. But suppose I were to consider **my tree's errors**. The fundamental idea of ***boosting*** is to start with a weak learner and then to use information about its errors to build a new model that can supplement the original model.

## Two Types

The two main types of boosting available in Scikit-Learn are adaptive boosting (AdaBoostClassifier, AdaBoostRegressor) and gradient boosting (GradientBoostingClassifier, GradientBoostingRegressor).

Again, the fundamental idea of boosting is to use a sequence of **weak** learners to build a model. Though the individual learners are weak, the idea is to train iteratively in order to produce a better predictor. More specifically, the first learner will be trained on the data as it stands, but future learners will be trained on modified versions of the data. The point of the modifications is to highlight the "hard-to-predict-accurately" portions of the data.

# Create Some Noisy Data

In [ ]:
n = 200
X = np.random.rand(n, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(n)

fig, ax = plt.subplots()
ax.scatter(X, y, alpha=0.3);

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [ ]:
f, (ax0,ax1) = plt.subplots(ncols=2, figsize=(12,4))
ax0.scatter(X_train, y_train, alpha=0.3)
ax1.scatter(X_test, y_test, alpha=0.3);

# AdaBoost (Adaptive Boosting)

- **AdaBoost** works by iteratively adapting two related series of weights, one attached to the datapoints and the other attached to the learners themselves. Datapoints that are incorrectly classified receive greater weights for the next learner in the sequence. That way, future learners will be more likely to focus on those datapoints. At the end of the sequence, the learners that make better predictions, especially on the datapoints that are more resistant to correct classification, receive more weight in the final "vote" that determines the ensemble's prediction. <br/> Suppose we have a binary classification problem and we represent the two classes with 1 and -1. (This is standard for describing the algorithm of AdaBoost.) <br/>
Then, in a nutshell: <br/>
    1. Train a weak learner. <br/>
    2. Calculate its error $\epsilon$. <br/>
    3. Use that error as a weight on the classifier: $\theta = \frac{1}{2}ln\left(\frac{1-\epsilon}{\epsilon}\right)$. <br/>
    Note that $\theta$ CAN be negative. This represents a classifier whose accuracy is _worse_ than chance. <br/>
    4. Use _that_ to adjust the data points' weights: $w_{n+1} = w_n\left(\frac{e^{\pm\theta}}{scaler}\right)$. Use $+\theta$ for incorrect predictions, $-\theta$ for correct predictions. <br/>  $\rightarrow$ For more detail on AdaBoost, see [here](https://towardsdatascience.com/boosting-algorithm-adaboost-b6737a9ee60c).

## Algorithm

- Train model
- Inflate errors
- Retrain model using errors (repeat)

![](images/adaboost.png)

## Voting

Combine weak learners

+ Vote by combining these calculated $y$ for each crossed-area (negative for "not blue" or whatever)

## AdaBoost in Scikit-Learn

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html

In [ ]:
model = AdaBoostRegressor()
model.fit(X_train, y_train)
model.predict(X_train)

In [ ]:
model.score(X_test,y_test)

#### Hyperparameters

```python
model = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth=4), n_estimators = 5)
```

`base_estimator`: The model utilized for the weak learners (Warning: Don't forget to import the model that you decide to use for the weak learner).
`n_estimators`: The maximum number of weak learners used.

In [ ]:
model = AdaBoostRegressor(
            base_estimator=DecisionTreeRegressor(max_depth=6), 
            n_estimators=200,
            random_state=27
)

model.fit(X_train, y_train)
model.score(X_test,y_test)

### Galaxy Data

In [ ]:
galaxies = pd.read_csv('COMBO17.csv')
galaxies.head()

This is a dataset about galaxies. The Mcz and MCzml columns are measures of redshift, which is our target. Mcz is usually understood to be a better measure, so that will be our target column. Many of the other columns have to do with various measures of galaxies' magnitudes. For more on the dataset, see [here](https://astrostatistics.psu.edu/datasets/COMBO17.html).

In [ ]:
galaxies.columns

In [ ]:
galaxies.isnull().sum().sum()

In [ ]:
galaxies.info()

In [ ]:
galaxies = galaxies.dropna()

Let's collect together the columns that have high correlation with Mcz, our target:

In [ ]:
preds = []
for ind in galaxies.corr()['Mcz'].index:
    if abs(galaxies.corr()['Mcz'][ind]) > 0.5:
        preds.append(ind)

In [ ]:
galaxies[preds].corr()

These various magnitude columns all have high correlations **with one another**! Let's try a simple model with just the S280MAG column, since it has the highest correlation with Mcz.

In [ ]:
galaxies_x = galaxies['S280MAG']
galaxies_y = galaxies['Mcz']

Since we only have one predictor, we can visualize the correlation with the target! We can also reshape it for modeling purposes!

In [ ]:
galaxies_x_rev = galaxies_x.values.reshape(-1, 1)

In [ ]:
fig, ax = plt.subplots()

ax.scatter(galaxies_x_rev, galaxies_y, alpha=0.1);

In [ ]:
galaxies_x_train, galaxies_x_test, galaxies_y_train, galaxies_y_test =\
train_test_split(galaxies_x_rev, galaxies_y, random_state=42)

In [ ]:
abr = AdaBoostRegressor(random_state=42)

abr.fit(galaxies_x_train, galaxies_y_train)

In [ ]:
cross_val_score(abr, galaxies_x_train, galaxies_y_train, cv=5)

#### Hyperparameter Tuning

Let's see if we can do better by trying different hyperparameter values:

In [ ]:
gs = GridSearchCV(estimator=abr,
                 param_grid={
                     'n_estimators': [25, 50, 100],
                     'loss': ['linear', 'square']
                 }, cv=5)

In [ ]:
gs.fit(galaxies_x_train, galaxies_y_train)

In [ ]:
gs.best_params_

# Gradient Boosting

> Use gradient descent to improve the model

![](images/gradient_boosting_residuals.png)

- **Gradient Boosting** works instead by training each new learner on the residuals of the model built with the learners that have so far been constructed. That is, Model $n+1$ (with $n+1$ learners) will focus on the predictions of Model $n$ (with only $n$ learners) that were **most off the mark**. As the training process repeats, the learners learn and the residuals get smaller. I would get a sequence going: <br/> Model 0 is very simple. Perhaps it merely predicts the mean: <br/>
$\hat{y}_0 = \bar{y}$; <br/>
Model 1's predictions would then be the sum of (i) Model 0's predictions and (ii) the predictions of the model fitted to Model 0's residuals: <br/> $\hat{y}_1 = \hat{y}_0 + \hat{(y - \hat{y})}_{err0}$; <br/>
Now iterate: Model 2's predictions will be the sum of (i) Model 0's predictions, (ii) the predictions of the model fitted to Model 0's residuals, and (iii) the predictions of the model fitted to Model 1's residuals: <br/> $\hat{y}_2 = \hat{y}_0 + \hat{(y - \hat{y})}_{err0} + \hat{(y - \hat{y})}_{err1}$<br/>
Etc.
<br/>

$\rightarrow$ How does gradient boosting work for a classification problem? How do we even make sense of the notion of a gradient in that context? The short answer is that we appeal to the probabilities associated with the predictions for the various classes. See more on this topic [here](https://sefiks.com/2018/10/29/a-step-by-step-gradient-boosting-example-for-classification/). <br/> $\rightarrow$ Why is this called "_gradient_ boosting"? Because using a model's residuals to build a new model is using information about the derivative of that model's loss function. See more on this topic [here](https://www.ritchievink.com/blog/2018/11/19/algorithm-breakdown-why-do-we-call-it-gradient-boosting/).

## Algorithm

Use mean squared error (MSE) and want to minimize that <-- done by gradient descent

Use the residuals (pattern in the residuals) to create an even better model

1. Fit a model to the data, $F_1(x) = y$
2. Fit a model to the residuals, $h_1(x) = y - F_1(x)$
3. Create a new model, $F_2(x) = F_1(x) + h_1(x)$
4. Repeat

## Example of Iterative Steps

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

> Parts adapted from https://github.com/ageron/handson-ml/blob/master/07_ensemble_learning_and_random_forests.ipynb

### Recall our noisy data from earlier

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X, y, alpha=0.3);

In [ ]:
f, (ax0, ax1) = plt.subplots(ncols=2, figsize=(12,4))
ax0.scatter(X_train, y_train, alpha=0.3)
ax1.scatter(X_test, y_test, alpha=0.3);

In [ ]:
f, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12,4))
ax0.scatter(X, y, alpha=0.3)
ax1.scatter(X_train, y_train, alpha=0.3)
ax2.scatter(X_test, y_test, alpha=0.3);

### Train iteratively on the residuals of its predecessor

In [ ]:
# First iteration
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=seed)
tree_reg1.fit(X_train, y_train)

In [ ]:
# Second iteration
y2 = y_train - tree_reg1.predict(X_train)
y2

In [ ]:
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=seed)
tree_reg2.fit(X_train, y2)

Let's make a function that will repeat this process for us

In [ ]:
def my_gradient_boosting_regressor(n_estimators, X, y, random_state=27):
    # Save the iteratively trained models and residuals
    trained_weak_learners = []
    past_residuals = []
    
    # Initial conditions
    new_y = y
    past_residuals.append(new_y)
    # Iteratively train model
    for n in range(n_estimators):
        # Train the new model on past residuals (note first model trains on y)
        new_model = DecisionTreeRegressor(max_depth=2, random_state=random_state)
        new_model.fit(X, new_y)
        # Find the new residuals (used to train the next model)
        new_y = new_y - new_model.predict(X) 
        # Save the (trained) model and the new residuals
        trained_weak_learners.append(new_model)
        past_residuals.append(new_y)
        
    return trained_weak_learners, past_residuals

In [ ]:
m, ys = my_gradient_boosting_regressor(3, X_train, y_train)

### Observe how the regressor gets better

In [ ]:
def plot_preds(regressors, X, y, axes, label=None, style='r-', data_style='b.',
               data_label=None, ax=None):
    x1 = np.linspace(axes[0], axes[1], 500)
    y_pred = sum(regressor.predict(x1.reshape(-1, 1)) for regressor in regressors)
    ax.plot(X[:, 0], y, data_style, label=data_label)
    ax.plot(x1, y_pred, style, linewidth=2, label=label)
    if label or data_label:
        ax.legend(loc='upper center')

def gradient_boost_and_plot(n_estimators, X, y):
    
    fig_size = (16, 4*n_estimators)
    model_list, resid_list = my_gradient_boosting_regressor(16, X, y)
    f, axes = plt.subplots(nrows=n_estimators, ncols=2, figsize=fig_size)

    base_label ='h(x_1) = '
    iterative_label = 'y'
    for i in range(n_estimators):
        # Next set of residuals
        y = resid_list[i]
        # Must be a list of one item
        past_model = [model_list[i]]
        # Includes current model
        past_models = model_list[:(i+1)]
        # New labels
        new_label = f'h_{i}(x_{i})'
        base_label = f'{base_label} + {new_label}'

        plot_preds(
            past_model,
            X,
            y,
            axes=[-0.5, 0.5, -0.1, 0.8], 
            label=f'${iterative_label}$',
            style="g-",
            data_label='Training set',
            ax=axes[i][0]
        )
        axes[i][0].set_title('Residuals and tree predictions')

        plot_preds(
            past_models,
            X,
            resid_list[0],
            axes=[-0.5, 0.5, -0.1, 0.8],
            label=f'${base_label}$',
            data_label='Training set',
            ax=axes[i][1]
        )
        axes[i][1].set_ylabel('$y$', fontsize=16, rotation=0)
        axes[i][1].set_title('Ensemble predictions')

        # Update labels for next round
        base_label = f'{base_label} + '
        iterative_label = f'{iterative_label} - {new_label}'
        
    return f, model_list, resid_list;

In [ ]:
gradient_boost_and_plot(3, X_train, y_train);

### Using SciKit-learn's Gradient Boosting

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X_train, y_train)

## Comparing gradient boosting with many estimators 

In [ ]:
# More estimators
gbrt_slow = GradientBoostingRegressor(max_depth=2,
                                      n_estimators=100,
                                      learning_rate=0.1,
                                      random_state=seed)
gbrt_slow.fit(X_train, y_train)

# Even more estimators
gbrt_slower = GradientBoostingRegressor(max_depth=2,
                                        n_estimators=1000,
                                        learning_rate=0.1,
                                        random_state=seed)
gbrt_slower.fit(X_train, y_train)

In [ ]:
f, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))

axes = [-0.5, 0.5, -0.1, 0.8]
plot_preds([gbrt], X, y, axes=axes, ax=ax0)
ax0.set_title(
    f"learning_rate={gbrt.learning_rate}, n_estimators={gbrt.n_estimators}",
              fontsize=14)

plot_preds([gbrt_slow], X, y, axes=axes, ax=ax1)
ax1.set_title(
    f"learning_rate={gbrt_slow.learning_rate}, n_estimators={gbrt_slow.n_estimators}",
              fontsize=14)

plot_preds([gbrt_slower], X, y, axes=axes, ax=ax2)
ax2.set_title(
    f"learning_rate={gbrt_slower.learning_rate}, n_estimators={gbrt_slower.n_estimators}",
              fontsize=14)

plt.tight_layout()

In [ ]:
gbrt.score(X_train, y_train), gbrt.score(X_test, y_test)

In [ ]:
gbrt_slow.score(X_train, y_train), gbrt_slow.score(X_test, y_test)

In [ ]:
gbrt_slower.score(X_train, y_train), gbrt_slower.score(X_test, y_test)

# XGBoost

From [XGBoost's documentation](https://xgboost.readthedocs.io/):

>_**XGBoost** is an optimized distributed gradient boosting library designed to be highly **efficient**, **flexible** and **portable**. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. The same code runs on major distributed environment (Hadoop, SGE, MPI) and can solve problems beyond billions of examples._

## XGBoost Regression

In [ ]:
grad_boost = xgboost.XGBRegressor(random_state=42, objective='reg:squarederror')

grad_boost.fit(X, y)

In [ ]:
cross_val_score(grad_boost, X, y, cv=5)

# Level Up: Regression or Classification?

What does my target look like?

In [ ]:
galaxies['Mcz'].hist();

There seems to be a bit of a bimodal shape here. We might therefore try predicting whether the redshift factor is likely to be greater or less than 0.5:

In [ ]:
galaxies['bool'] = galaxies['Mcz'] > 0.5

In [ ]:
galaxies.tail()

In [ ]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(galaxies_x_rev, galaxies['bool'])

## Adaboost Classification

In [ ]:
abc = AdaBoostClassifier(random_state=42)

abc.fit(x_train2, y_train2)

In [ ]:
abc.score(x_test2, y_test2)

In [ ]:
precision_score(y_test2, abc.predict(x_test2))

In [ ]:
recall_score(y_test2, abc.predict(x_test2))

## GradientBoosting

In [ ]:
gbc = GradientBoostingClassifier(random_state=42)

gbc.fit(x_train2, y_train2)

In [ ]:
gbc.score(x_test2, y_test2)

In [ ]:
precision_score(y_test2, gbc.predict(x_test2))

In [ ]:
recall_score(y_test2, gbc.predict(x_test2))

In [ ]:
confusion_matrix(y_test2, gbc.predict(x_test2))

## XGBoost Classification

In [ ]:
grad_boost_class = xgboost.XGBClassifier(random_state=42, objective='binary:logistic')

grad_boost_class.fit(x_train2, y_train2)

In [ ]:
grad_boost_class.score(x_test2, y_test2)